# Web Scraping con Beautiful Soup.

* * * 

### Iconos utilizados en este cuaderno
🔔 **Pregunta**: Una pregunta rápida para ayudarte a entender lo que está pasando.<br>
🥊 **Desafío**: Ejercicio interactivo. ¡Trabajaremos en estos durante el taller!<br>
⚠️ **Advertencia**: Aviso sobre cosas complicadas o errores comunes.<br>
💡 **Consejo**: Cómo hacer algo de manera más eficiente o efectiva.<br>
🎬 **Demostración**: Mostrar algo más avanzado, para que sepas para qué se puede usar Python.<br>

### Objetivos de aprendizaje
1. [Reflexión: ¿Raspado o no raspado?](#when)
2. [Extracción y análisis de HTML](#extract)
3. [Raspado de la Asamblea General de Illinois](#scrape)


<a id='when'></a>

# Scrape o no Scrape

Cuando queremos acceder a datos de la web, primero debemos asegurarnos de si el sitio web que nos interesa ofrece una API web. Plataformas como Twitter, Reddit y el New York Times ofrecen APIs. **Consulta el taller de APIs web de [Python de D-Labs](https://github.com/dlab-berkeley/Python-Web-APIs) si quieres aprender a usar APIs.**

Sin embargo, a menudo hay casos en los que no existe una API web. En estos casos, podemos recurrir al web scraping, donde extraemos el HTML subyacente de una página web y obtenemos directamente la información que queremos. Hay varios paquetes en Python que podemos usar para realizar estas tareas. Nos centraremos en dos paquetes: Requests y Beautiful Soup.

Nuestro estudio de caso será raspar información sobre los [senadores estatales de Illinois](http://www.ilga.gov/senate), así como la [lista de proyectos de ley](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) que cada senador ha patrocinado. Antes de comenzar, revisa estos sitios web para ver su estructura.




## Instalación

Usaremos dos paquetes principales: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) y [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Adelante, instala estos paquetes si aún no lo has hecho:


In [ ]:
%pip install requests

In [ ]:
%pip install beautifulsoup4

También instalaremos el paquete `lxml` que ayuda a soportar parte del análisis que realiza Beautiful Soup:

In [ ]:
%pip install lxml

In [ ]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracción y Análisis de HTML 

Para realizar correctamente la extracción y el análisis de HTML, seguiremos los siguientes 4 pasos:
1. Realizar una solicitud GET
2. Analizar la página con Beautiful Soup
3. Buscar elementos HTML
4. Obtener atributos y texto de estos elementos

## Paso 1: Realizar una Solicitud GET para Obtener el HTML de una Página

Podemos usar la biblioteca Requests para:

1. Hacer una solicitud GET a la página, y  
2. Leer el código HTML de la página web.

El proceso de realizar una solicitud y obtener un resultado se asemeja al flujo de trabajo de una API web. Sin embargo, en este caso, estamos haciendo una solicitud directamente al sitio web, y tendremos que analizar el HTML por nuestra cuenta. Esto contrasta con recibir datos organizados en un formato más sencillo como `JSON` o `XML`.


In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

## Paso 2: Analizar la Página con Beautiful Soup

Ahora utilizamos la función `BeautifulSoup` para analizar la respuesta y convertirla en un árbol HTML. Esto devuelve un objeto (llamado **soup object**) que contiene todo el HTML del documento original.

Si encuentras un error relacionado con una biblioteca de análisis, asegúrate de haber instalado el paquete `lxml` para proporcionar a Beautiful Soup las herramientas necesarias para el análisis.

In [ ]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

La salida se ve bastante similar a la anterior, pero ahora está organizada en un objeto `soup`, lo que nos permite recorrer la página con mayor facilidad.


## Paso 3: Buscar Elementos HTML

Beautiful Soup ofrece varias funciones para encontrar componentes útiles en una página. Permite buscar elementos utilizando:

1. Etiquetas HTML  
2. Atributos HTML  
3. Selectores CSS  

Primero, busquemos **etiquetas HTML**.  

La función `find_all` busca en el árbol de `soup` todos los elementos con una etiqueta HTML específica y devuelve todos esos elementos.

**¿Qué hace el siguiente ejemplo?**



In [ ]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

Dado que `find_all()` es el método más popular en la API de búsqueda de Beautiful Soup, existe un atajo para usarlo. Si tratas al objeto `BeautifulSoup` como si fuera una función, es equivalente a llamar a `find_all()` en ese objeto.

Estas dos líneas de código son equivalentes:

In [ ]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

¿Cuántos enlaces obtuvimos?

In [ ]:
print(len(a_tags))

¡Eso es mucho! Muchos elementos en una página tendrán la misma etiqueta HTML. Por ejemplo, si buscas todo lo que tenga la etiqueta `a`, probablemente obtendrás muchos resultados, muchos de los cuales quizás no quieras. Recuerda que la etiqueta `a` define un hipervínculo, por lo que generalmente encontrarás muchos en cualquier página.

¿Qué pasa si queremos buscar etiquetas HTML con ciertos atributos, como clases CSS específicas?

Podemos hacerlo agregando un argumento adicional a la función `find_all`. En el ejemplo siguiente, estamos encontrando todas las etiquetas `a` y luego filtrando aquellas con `class_="sidemenu"`:


In [ ]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

Una forma más eficiente de buscar elementos en un sitio web es a través de un **selector CSS**. Para esto, debemos usar un método diferente llamado `select()`. Solo tienes que pasar una cadena al método `.select()` para obtener todos los elementos que coincidan con ese selector CSS válido.

En el ejemplo anterior, podemos usar `"a.sidemenu"` como selector CSS, lo que devuelve todas las etiquetas `a` con la clase `sidemenu`.



In [ ]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

## 🥊 Desafío: Encontrar Todos

Usa BeautifulSoup para encontrar todos los elementos `a` con la clase `mainmenu`.

In [ ]:
# YOUR CODE HERE


## Paso 4: Obtener Atributos y Texto de los Elementos

Una vez que identificamos los elementos, queremos acceder a la información dentro de ellos. Generalmente, esto significa dos cosas:

1. Texto  
2. Atributos  

Obtener el texto dentro de un elemento es fácil. Todo lo que tenemos que hacer es usar el miembro `text` de un objeto `tag`:

In [ ]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.sidemenu")

# Examine the first link
first_link = side_menu_links[0]
print(first_link)

# What class is this variable?
print('Class: ', type(first_link))

¡Es una etiqueta de Beautiful Soup! Esto significa que tiene un miembro `text`:

In [ ]:
print(first_link.text)

A veces queremos obtener el valor de ciertos atributos. Esto es particularmente relevante para las etiquetas `a`, o enlaces, donde el atributo `href` nos indica a dónde lleva el enlace.

💡 **Consejo**: Puedes acceder a los atributos de una etiqueta tratándola como un diccionario:

In [ ]:
print(first_link['href'])

## 🥊 Desafío: Extraer atributos específicos

Extrae todos los atributos `href` de cada URL con la clase `mainmenu`.

In [ ]:
# YOUR CODE HERE


<a id='scrape'></a>

# Arrastrando la Asamblea General de Illinois
Aunque parezca increíble, estas son las herramientas fundamentales para arrastrar un sitio web. Una vez que dediques más tiempo a familiarizarte con HTML y CSS, simplemente será cuestión de comprender la estructura de un sitio web en particular y aplicar inteligentemente las herramientas de Beautiful Soup y Python.

Apliquemos estas habilidades para arrastrar la [98.ª Asamblea General de Illinois](http://www.ilga.gov/senate/default.asp?GA=98)..

En concreto, nuestro objetivo es arrastrar información sobre cada senador, incluyendo su nombre, distrito y partido.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Search for the Table Elements

Our goal is to obtain the elements in the table on the webpage. Remember: rows are identified by the `tr` tag. Let's use `find_all` to obtain these elements.

In [ ]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)

⚠️ **Warning**: Keep in mind: `find_all` gets *all* the elements with the `tr` tag. We only want some of them. If we use the 'Inspect' function in Google Chrome and look carefully, then we can use some CSS selectors to get just the rows we're interested in. Specifically, we want the inner rows of the table:

In [ ]:
# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

It looks like we want everything after the first two rows. Let's work with a single row to start, and build our loop from there.

In [ ]:
example_row = rows[2]
print(example_row.prettify())

Let's break this row down into its component cells/columns using the `select` method with CSS selectors. Looking closely at the HTML, there are a couple of ways we could do this.

* We could identify the cells by their tag `td`.
* We could use the the class name `.detail`.
* We could combine both and use the selector `td.detail`.

In [ ]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

We can confirm that these are all the same.

In [ ]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Let's use the selector `td.detail` to be as specific as possible.

In [ ]:
# Select only those 'td' tags with class 'detail' 
detail_cells = example_row.select('td.detail')
detail_cells

Most of the time, we're interested in the actual **text** of a website, not its tags. Recall that to get the text of an HTML element, we use the `text` member:

In [ ]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

Looks good! Now we just use our basic Python knowledge to get the elements of this list that we want. Remember, we want the senator's name, their district, and their party.

In [ ]:
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

## Getting Rid of Junk Rows

We saw at the beginning that not all of the rows we got actually correspond to a senator. We'll need to do some cleaning before we can proceed forward. Take a look at some examples:

In [ ]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

When we write our for loop, we only want it to apply to the relevant rows. So we'll need to filter out the irrelevant rows. The way to do this is to compare some of these to the rows we do want, see how they differ, and then formulate that in a conditional.

As you can imagine, there a lot of possible ways to do this, and it'll depend on the website. We'll show some here to give you an idea of how to do this.

In [ ]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

Perhaps good rows have a length of 5. Let's check:

In [ ]:
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

We found a footer row in our list that we'd like to avoid. Let's try something else:

In [ ]:
rows[2].select('td.detail') 

In [ ]:
# Bad row
print(rows[-1].select('td.detail'), '\n')

# Good row
print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

Looks like we found something that worked!

## Enlazar todo

Ahora que hemos visto cómo obtener los datos que queremos de una fila, así como filtrar las filas que no queremos, vamos a juntarlo todo en un bucle.

In [ ]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

In [ ]:
# Should be 61
len(members)

Echemos un vistazo a lo que tenemos en  `members`.

In [ ]:
print(members[:5])

## 🥊  Desafío: Obtener elementos `href` que apunten a los proyectos de ley de los miembros

El código anterior recupera información sobre:
- el nombre del senador
- su número de distrito,
- y su partido.

Ahora queremos recuperar la URL de la lista de proyectos de ley de cada senador. Cada URL seguirá un formato específico. 

El formato de la lista de proyectos de ley para un senador determinado es:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

para obtener algo como:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

en el que `MEMBER_ID=1911`. 

Debería poder ver que, desafortunadamente, `MEMBER_ID` ` no es actualmente algo que se extraiga en nuestro código de raspado.

Tu tarea inicial es modificar el código anterior para que también **recuperemos la URL completa que apunta a la página correspondiente de los proyectos de ley patrocinados por las primarias**, para cada miembro, y la devolvamos junto con su nombre, distrito y partido.

Consejos: 

* Para ello, querrás obtener el elemento de anclaje apropiado (`<a>`) en cada fila de legisladores de la tabla. Puedes utilizar de nuevo el método `.select()` en el objeto `row` en el bucle para hacer esto - similar al comando que encuentra todas las celdas `td.detail` en la fila. Recuerda que sólo queremos el enlace a los proyectos de ley del legislador, no a los comités ni a la página de perfil del legislador.
* El HTML de los elementos de anclaje tendrá el siguiente aspecto: `<a href="/senate/Senator.asp/...">Bills</a>`. La cadena en el atributo `href` contiene el enlace **relative** que buscamos. Puedes acceder a un atributo de un objeto BeatifulSoup `Tag` de la misma forma que accedes a un diccionario de Python: `anchor['attributeName']`. Véase el <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> para más detalles.
* Hay un montón de maneras diferentes de utilizar BeautifulSoup para hacer las cosas. lo que usted necesita hacer para sacar el `href` está bien.

El código ha sido parcialmente rellenado para usted. Rellénalo donde dice `#YOUR CODE HERE`. Guarda la ruta en un objeto llamado `full_path`.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [ ]:
# Uncomment to test 
# members[:5]

## 🥊  Desafío: Modularice su código

Convierta el código anterior en una función que acepte una URL, rastree la URL en busca de sus senadores y devuelva una lista de tuplas con información sobre cada senador.

In [ ]:
# YOUR CODE HERE
def get_members(url):
    return [___]


In [ ]:
# Test your code
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

## 🥊 Reto para llevar a casa: Escribir una función rascadora

Queremos raspar las páginas web correspondientes a los proyectos de ley patrocinados por cada uno de ellos.
Escribe una función llamada `get_bills(url)` para analizar una URL de facturas dada. Esto implicará:

  - solicitando la URL mediante la biblioteca <a href="http://docs.python-requests.org/en/latest/">`requests`</a>   - utilizando las características de la biblioteca `BeautifulSoup` para encontrar todos los elementos `<td>` con la clase `billlist`.
  - devuelve una _lista_ de tuplas, cada una con:
      - descripción (2ª columna)
      - cámara (S o H) (3ª columna)
      - la última acción (4ª columna)
      - la fecha de la última acción (5ª columna)
      
Esta función se ha completado parcialmente. Rellene el resto.

In [ ]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # YOUR CODE HERE
        bill_id =
        description =
        chamber =
        last_action =
        last_action_date =
        bill = (bill_id, description, chamber, last_action, last_action_date)
        bills.append(bill)
    return bills

In [ ]:
# Uncomment to test your code
# test_url = senate_members[0][3]
# get_bills(test_url)[0:5]

### Rechazar todas las facturas

Por último, crea un diccionario `bills_dict` que asigne un número de distrito (la clave) a una lista de proyectos de ley (el valor) procedentes de ese distrito. Puedes hacerlo recorriendo todos los miembros del senado en `members_dict` y llamando a `get_bills()` para cada una de sus URLs de proyectos de ley asociadas.

**NOTA:** por favor llama a la función `time.sleep(1)` en cada iteración del bucle, para que no destruyamos la web del estado.

In [ ]:
# YOUR CODE HERE


In [ ]:
# Uncomment to test your code
# bills_dict[52]